In [1]:
import pandas as pd
import re
from mlxtend.frequent_patterns import apriori, association_rules
data = pd.read_csv("Team_Sports_Survey_Preprocessed.csv")
# remove the eg. for each items
def remove_parentheses(text):
    if isinstance(text, str):  
        return re.sub(r"\s*\(.*?\)", "", text)
    return text  

In [2]:
# Get the rows which preferred time is Afternoon
# Split the values into a single items in the "Preferred Check-in Days" column
data["Preferred Time for Swimming Pool Activities"] = data["Preferred Time for Swimming Pool Activities"].str.split(", ")
# Get only rows where Saturday or Sunday is in the dataset
time_data = data[data["Preferred Time for Swimming Pool Activities"].apply(lambda x: any(time in ["Afternoon (12PM-5PM)"] for time in x))]

In [3]:
# Split the values into a single items
time_data["Preferred Swimming Pool Activities"] = time_data["Preferred Swimming Pool Activities"].apply(remove_parentheses).str.split(",")

# Convert the items into a transactional data 
transactions = time_data["Preferred Swimming Pool Activities"].explode().str.strip().reset_index() 
transactions_encoded = transactions.pivot_table(index="index", columns="Preferred Swimming Pool Activities", aggfunc=lambda x: 1, fill_value=0) 

C:\Users\joaqu\AppData\Local\Temp\ipykernel_7484\4294322917.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_data["Preferred Swimming Pool Activities"] = time_data["Preferred Swimming Pool Activities"].apply(remove_parentheses).str.split(",")


In [4]:
# Use Apriori algorithm to find frequent association
frequent_items= apriori(transactions_encoded, min_support=0.2, use_colnames=True)
rule = association_rules(frequent_items)
rule[['antecedents','consequents','support', 'confidence', 'lift']].sort_values("confidence", ascending=False)

C:\Users\joaqu\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,support,confidence,lift
2,"(Play with Pool Toys, Practice Actual Swimming...",(Relaxing in Pool),0.228571,1.000000,1.129032
0,(Play with Pool Toys),(Relaxing in Pool),0.371429,0.928571,1.048387
1,(Practice Actual Swimming Techniques),(Relaxing in Pool),0.457143,0.842105,0.950764
